# Merge samples into one dataset

## First: Set parameters

**error**: ppm tolerance for the deviation between m/z values of the same peak.

**minsamples**: the minimum number of occurances of a given peak to be included in the aligned table.

In [ ]:
error=1
minsamples=2

## Second: Upload the file into the *input* folder and run the following cell

In [ ]:
from merge import *

files = os.listdir("input")
for file in list(files):
    if not file.endswith(".xlsx"):
        files.remove(file)
        
for file in files:
    
    file = "input/"+file

    xl = pd.ExcelFile(file)
    
    #### Define error and min samples #####
    error=1
    minsamples=2
    #######################################

    folder = "output/" + os.path.splitext(os.path.split(file)[1])[0]
    
    try:
        os.mkdir(folder)
    except:
        pass
    base_name = os.path.splitext(os.path.split(file)[1])[0]
    name = folder+'/'+base_name
    for sheet in xl.sheet_names:
        print("\nAligning " + sheet +' at ' + str(error) + (' ppm'))
        if len(pd.read_excel(file).columns)==2 or len(pd.read_excel(file).columns)==1:
            print("\nOnly 1 sample, no aligment needed...")
        else:
            print ('Starting alingment...')
            df = split_sample_label(merge(prep_table(file), error=error))
            #Grava alinhado raw em .csv
            df.swaplevel(axis=1).to_csv(name+'_aligned_'+sheet+"_"+str(error)+'ppm.csv', index=True, header=True)
            cleaned = cleandf(df, minsamples=minsamples)
            cleaned.to_csv(name+'_aligned_'+sheet+"_"+str(error)+'ppm_cleaned_'+str(minsamples)+'_min_samples'+'.csv', index=True, header=True)
            print (sheet +' aligned at ' + str(error) + (' ppm'))
            # round to 7 decimal places to match masstrix and sort the dataframe for futher analisys
            cleaned.columns = cleaned.columns.droplevel(1)
            cleaned = cleaned.reset_index()
            cleaned['index'] = round(cleaned['index'], 7)
    print()
    print('All Finished!')
